Berdasarkan isu [#99](https://github.com/taruma/hidrokit/issues/99): **buat perhitungan curah hujan dengan metode poligon Thiessen**

Referensi isu:
- Limantara, Lily M. (2018). _Rekayasa Hidrologi_. Penerbit Andi, Yogyakarta. (hal. 57).

Deskripsi Permasalahan:
- Menghitung nilai hujan rata-rata daerah menggunakan metode poligon Thiessen

# PERSIAPAN DAN DATASET

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
try:
    import hidrokit
except ModuleNotFoundError:
    !pip install git+https://github.com/taruma/hidrokit.git@latest -q
    import hidrokit
print(f'hidrokit version: {hidrokit.__version__}')

hidrokit version: 0.3.5-beta.3


In [ ]:
!wget -O sample.xlsx "https://taruma.github.io/assets/hidrokit_dataset/data_daily_sample.xlsx" -q
dataset_path = 'sample.xlsx'

In [ ]:
from hidrokit.contrib.taruma import hk88

_data = hk88.read_workbook(dataset_path, ['STA_A', 'STA_B', 'STA_C'], 
                           as_df=False)
dataset = pd.concat(_data, sort=True, axis=1).infer_objects()
dataset.info()
dataset.head()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5478 entries, 2001-01-01 to 2015-12-31
Freq: D
Data columns (total 3 columns):
STA_A    5477 non-null float64
STA_B    5470 non-null float64
STA_C    5475 non-null float64
dtypes: float64(3)
memory usage: 171.2 KB


,STA_A,STA_B,STA_C
2001-01-01,0.0,0.0,0.00
2001-01-02,0.0,0.0,0.65
2001-01-03,0.0,45.0,9.16
2001-01-04,0.0,0.0,0.00
2001-01-05,0.0,5.0,1.03


# KODE

In [ ]:
def thiessen_weight(area):
    area_val = list(area.values())
    area_percent = area_val / np.sum(area_val)
    key = list(area.keys())
    return dict(zip(key, area_percent))

def apply_thiessen(dataset, area, columns=None, as_df=True):
    weight = thiessen_weight(area)

    columns = columns if columns is not None else dataset.columns

    val = []
    for col in columns:
        val.append(dataset[col].values * weight[col])
    
    np_val = np.stack(val, axis=1)

    if as_df:
        return pd.DataFrame(
            data=np_val.sum(axis=1), index=dataset.index, columns=['thiessen']
        )
    else:
        return np_val.sum(axis=1)

# FUNGSI

## Fungsi `thiessen_weight()`

Fungsi ini menerima input _dictionary_ yang berisikan besar area setiap stasiun dan memberi keluaran bobotnya dalam bentuk _dictionary_. 

In [ ]:
AREA_BASIN = {
    'STA_A': 49.54,
    'STA_B': 144.39,
    'STA_C': 0
}

weight = thiessen_weight(AREA_BASIN)
weight

{'STA_A': 0.2554529985046151, 'STA_B': 0.744547001495385, 'STA_C': 0.0}

## Fungsi `apply_thiessen()`

Menerapkan perhitungan poligon thiessen dengan bobot masing-masing stasiun dan mengeluarkan hasil penjumlahannya.

Fungsi ini memiliki beberapa argumen opsional yaitu `as_df` dan `columns`.

In [ ]:
apply_thiessen(dataset, AREA_BASIN)

,thiessen
2001-01-01,0.000000
2001-01-02,0.000000
2001-01-03,33.504615
2001-01-04,0.000000
2001-01-05,3.722735
...,...
2015-12-27,14.174496
2015-12-28,29.190017
2015-12-29,1.788171
2015-12-30,1.277265


### Argumen `as_df=True`

Argumen ini memberi keluaran dalam bentuk `pandas.DataFrame` jika `as_df=True` (_default_), dan dalam bentuk `numpy.array` jika `as_df=False`.

In [ ]:
apply_thiessen(dataset, AREA_BASIN, as_df=False)

array([ 0.        ,  0.        , 33.50461507, ...,  1.78817099,
        1.27726499,  0.766359  ])

### Argumen `columns=None`

Argumen ini memastikan kolom mana saja yang dihitung. Jika tidak diisi, maka fungsi memproses seluruh kolom. Jika nilai kolom tidak tersedia dalam `area`, maka akan menghasilkan error. Gunakan opsi `columns` jika dataset berisikan kolom yang tidak digunakan dalam proses perhitungan thiessen.

In [ ]:
apply_thiessen(dataset, AREA_BASIN, columns=['STA_A', 'STA_C'])

,thiessen
2001-01-01,0.000000
2001-01-02,0.000000
2001-01-03,0.000000
2001-01-04,0.000000
2001-01-05,0.000000
...,...
2015-12-27,9.707214
2015-12-28,5.364513
2015-12-29,1.788171
2015-12-30,1.277265


# Changelog

```
- 20191217 - 1.0.0 - Initial
```

#### Copyright &copy; 2019 [Taruma Sakti Megariansyah](https://taruma.github.io)

Source code in this notebook is licensed under a [MIT License](https://choosealicense.com/licenses/mit/). Data in this notebook is licensed under a [Creative Common Attribution 4.0 International](https://creativecommons.org/licenses/by/4.0/). 
